# Numerische Methoden für Differentialgleichungen
## Aufgabenblatt 6
*gelöst von Stollmayer Max*

Zu implementieren ist eine Finite-Differenzen-Methode für die Poisson-Gleichung:

$$\begin{cases}
    -\Delta(x,y)=f(x,y),\quad (x,y)\in\Omega=[0,1]\times[0,1] \\
    u(x,y)=v(x,y),\quad (x,y)\in\partial\Omega
\end{cases}$$

wobei $f(x,y)=sin(x\cdot y)\cdot(x^2+y^2)$, und $v(x,y)=sin(x\cdot y)$.

In [3]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = [11, 6]
plt.rcParams["font.size"] = 12
plt.rcParams["axes.edgecolor"] = "white"
plt.rcParams["legend.frameon"] = True
plt.rcParams["legend.framealpha"] = 0.75
plt.rcParams["legend.facecolor"] = "white"
plt.rcParams["legend.edgecolor"] = "white"

In [4]:
def error(exact_vals, approx_vals):
    '''Maxmimum norm'''
    return max(np.abs(exact_vals - approx_vals))

def order(error, step_size):
    '''Approximation of order of convergence'''
    return np.log(error) / np.log(step_size)


def f(x,y):
    return np.sin(x*y) * (x**2 + y**2)

def v(x,y):
    return np.sin(x*y)

def exact(x,y):
    return v(x,y)

a = c = 0
b = d = 1
steps = 2**np.arange(3,8)
    
vals = ()
errors = ()
rates = ()

'''
for N in steps:
h = (b - a) / N
grid = a + h * np.arange(N+1)

u = FDM(f, grid, ua, ub)
vals += (u,)

e = error(u, exact(grid))
errors += (e,)
rates += (order(e, h),)

plt.plot(grid, u, label=f"{N} steps")

rates = order(errors,(b-a)/steps)
print("steps | error    | rate")
for N, e, r in zip(steps, errors, rates):
    print(f"  {N:3.0f} | {e:.2E} |   {r:2.0f}")

x = np.linspace(0, 2, 101)
plt.plot(x, exact(x), "k", label="exact")
plt.legend()
plt.show()'''

'\nfor N in steps:\nh = (b - a) / N\ngrid = a + h * np.arange(N+1)\n\nu = FDM(f, grid, ua, ub)\nvals += (u,)\n\ne = error(u, exact(grid))\nerrors += (e,)\nrates += (order(e, h),)\n\nplt.plot(grid, u, label=f"{N} steps")\n\nrates = order(errors,(b-a)/steps)\nprint("steps | error    | rate")\nfor N, e, r in zip(steps, errors, rates):\n    print(f"  {N:3.0f} | {e:.2E} |   {r:2.0f}")\n\nx = np.linspace(0, 2, 101)\nplt.plot(x, exact(x), "k", label="exact")\nplt.legend()\nplt.show()'